In [14]:
import casadi as ca

import sympy as sp


In [ ]:
L0 = 0.158
L1 = 0.78
L2 = 0.73
dt = 0.01
dead_time = 0.2
controller_dt = 0.01
l1=L1 / 2
l2=L2 / 2
M1=11.41
M2=50.14
I1=0.35
I2=0.25
c1=0
c2=0
g=9.8
Q = ca.diag([5000,8000,7000,8500])
R = ca.diag([0.02,0.01])
N = 400
horizon_dt = 0.01
x = [0.0876,0,0,0]




In [27]:
import sympy as sp

# シンボリック変数の定義
L1_sym, L2_sym = sp.symbols('L1 L2')  # アームの長さ
M1_sym, M2_sym = sp.symbols('M1 M2')  # 質量
I1_sym, I2_sym = sp.symbols('I1 I2')  # 慣性モーメント
G_sym = sp.symbols('G')              # 重力加速度
theta1, theta2 = sp.symbols('theta1 theta2')  # 角度
theta1_dot, theta2_dot = sp.symbols('theta1_dot theta2_dot')  # 角速度
u1, u2 = sp.symbols('u1 u2')  # 入力トルク

# 状態ベクトル
state = sp.Matrix([theta1, theta2, theta1_dot, theta2_dot])
u = sp.Matrix([u1, u2])

# αの計算
alpha1 = 1/4 * L1_sym**2 * M1_sym + I1_sym
alpha2 = 1/4 * L2_sym**2 * M2_sym + I2_sym
alpha3 = 1/2 * L1_sym * L2_sym * M2_sym * sp.cos(theta2)
alpha4 = L1_sym**2 * M2_sym

# 質量行列の逆行列計算
det_M = alpha2 * (alpha1 + alpha4) - alpha3**2
M_inv = sp.Matrix([
    [alpha2 / det_M, -(alpha2 + alpha3) / det_M],
    [-(alpha2 + alpha3) / det_M, (alpha1 + alpha2 + 2 * alpha3 + alpha4) / det_M]
])

# f_1とf_2の計算
f_1 = (
    u1 + 1/2 * L1_sym * L2_sym * M2_sym * sp.sin(theta2) * theta2_dot**2 -
    L1_sym * L2_sym * M2_sym * sp.sin(theta2) * theta1_dot * theta2_dot +
    1/2 * G_sym * L2_sym * M2_sym * sp.sin(theta1 + theta2) +
    1/2 * G_sym * L1_sym * M1_sym * sp.sin(theta1) -
    G_sym * L1_sym * M2_sym * sp.sin(theta1)
)

f_2 = (
    u2 - 1/2 * L1_sym * theta1_dot**2 * sp.sin(theta2) +
    1/2 * G_sym * L2_sym * M2_sym * sp.sin(theta1 + theta2)
)

f = sp.Matrix([f_1, f_2])

# 加速度の計算
ddot_theta = M_inv * f

# ヤコビアンの計算
jacobian_ddot_theta = ddot_theta.jacobian(state)

# 数値代入
values = {
    L1_sym: 0.78,
    L2_sym: 0.73,
    M1_sym: 11.41,
    M2_sym: 50.14,
    I1_sym: 0.35,
    I2_sym: 0.25,
    G_sym: 9.8,
    theta1: 0.0876,
    theta2: 0.0,
    theta1_dot: 0.0,
    theta2_dot: 0.0,
    u1: 0.0,
    u2: 0.0
}

# ヤコビアンの計算結果
jacobian_ddot_theta_numeric = jacobian_ddot_theta.subs(values).evalf()

# 結果表示
print(jacobian_ddot_theta_numeric)


Matrix([[-221.718850794638, -115.515512609623, 0, 0], [704.217510196917, 379.246622182072, 0, 0]])


In [29]:
jacobian_ddot_theta_numeric

Matrix([
[-221.718850794638, -115.515512609623, 0, 0],
[ 704.217510196917,  379.246622182072, 0, 0]])

In [32]:
jacobian_ddot_theta_numeric_simplified

Matrix([
[-221.718850794638, -115.515512609623, 0, 0],
[ 704.217510196917,  379.246622182072, 0, 0]])

In [28]:
jacobian_ddot_theta

Matrix([
[                                                           0.5*G*L2*M2*(-I2 - 0.5*L1*L2*M2*cos(theta2) - 0.25*L2**2*M2)*cos(theta1 + theta2)/(-0.25*L1**2*L2**2*M2**2*cos(theta2)**2 + (I2 + 0.25*L2**2*M2)*(I1 + 0.25*L1**2*M1 + L1**2*M2)) + (I2 + 0.25*L2**2*M2)*(0.5*G*L1*M1*cos(theta1) - G*L1*M2*cos(theta1) + 0.5*G*L2*M2*cos(theta1 + theta2))/(-0.25*L1**2*L2**2*M2**2*cos(theta2)**2 + (I2 + 0.25*L2**2*M2)*(I1 + 0.25*L1**2*M1 + L1**2*M2)),                                                                                                                                                                                                                                                                                                                                                                                                                                -0.5*L1**2*L2**2*M2**2*(I2 + 0.25*L2**2*M2)*(0.5*G*L1*M1*sin(theta1) - G*L1*M2*sin(theta1) + 0.5*G*L2*M2*sin(theta1 + theta2) - L1*L2*M

In [30]:
import sympy as sp

# パラメータ設定
L0 = 0.158
L1_val = 0.78
L2_val = 0.73
M1_val = 11.41
M2_val = 50.14
I1_val = 0.35
I2_val = 0.25
g_val = 9.8
x = [0.0876, 0, 0, 0]  # 初期状態

# αの定義
alpha1_sym = sp.symbols('alpha1')
alpha2_sym = sp.symbols('alpha2')
alpha3_sym = sp.symbols('alpha3') * sp.cos(sp.symbols('theta2'))
alpha4_sym = sp.symbols('alpha4')
alpha5_sym, alpha6_sym, alpha7_sym, alpha8_sym, alpha9_sym, alpha10_sym = sp.symbols('alpha5 alpha6 alpha7 alpha8 alpha9 alpha10')

# シンボリック変数の定義
theta1, theta2 = sp.symbols('theta1 theta2')  # 角度
theta1_dot, theta2_dot = sp.symbols('theta1_dot theta2_dot')  # 角速度
u1, u2 = sp.symbols('u1 u2')  # 入力トルク

# 状態ベクトル
state = sp.Matrix([theta1, theta2, theta1_dot, theta2_dot])
u = sp.Matrix([u1, u2])

# 質量行列の逆行列計算
det_M = alpha2_sym * (alpha1_sym + alpha4_sym) - alpha3_sym**2
M_inv = sp.Matrix([
    [alpha2_sym / det_M, -(alpha2_sym + alpha3_sym) / det_M],
    [-(alpha2_sym + alpha3_sym) / det_M, (alpha1_sym + alpha2_sym + 2 * alpha3_sym + alpha4_sym) / det_M]
])

# f_1とf_2の計算
f_1 = (
    u1 + alpha5_sym * sp.sin(theta2) * theta2_dot**2 -
    alpha6_sym * sp.sin(theta2) * theta1_dot * theta2_dot +
    alpha7_sym * sp.sin(theta1 + theta2) - alpha8_sym * sp.sin(theta1)
)

f_2 = (
    u2 - alpha9_sym * theta1_dot**2 * sp.sin(theta2) +
    alpha10_sym * sp.sin(theta1 + theta2)
)

f = sp.Matrix([f_1, f_2])

# 加速度の計算
ddot_theta = M_inv * f

# ヤコビアンの計算
jacobian_ddot_theta_simplified = ddot_theta.jacobian(state)

# αの数値計算
alpha1_val = 1/4 * L1_val**2 * M1_val + I1_val
alpha2_val = 1/4 * L2_val**2 * M2_val + I2_val
alpha3_val = 1/2 * L1_val * L2_val * M2_val * sp.cos(0.0)  # 初期状態のtheta2を代入
alpha4_val = L1_val**2 * M2_val
alpha5_val = 1/2 * L1_val * L2_val * M2_val
alpha6_val = L1_val * L2_val * M2_val
alpha7_val = 1/2 * L2_val * M2_val * g_val
alpha8_val = 1/2 * g_val * L1_val * (2 * M2_val - M1_val)
alpha9_val = 1/2 * L1_val
alpha10_val = 1/2 * L2_val * M2_val * g_val

# 数値代入辞書
values = {
    alpha1_sym: alpha1_val,
    alpha2_sym: alpha2_val,
    alpha3_sym: alpha3_val,
    alpha4_sym: alpha4_val,
    alpha5_sym: alpha5_val,
    alpha6_sym: alpha6_val,
    alpha7_sym: alpha7_val,
    alpha8_sym: alpha8_val,
    alpha9_sym: alpha9_val,
    alpha10_sym: alpha10_val,
    theta1: x[0],
    theta2: x[1],
    theta1_dot: x[2],
    theta2_dot: x[3],
    u1: 0.0,
    u2: 0.0
}

# 数値代入と評価
jacobian_ddot_theta_numeric_simplified = jacobian_ddot_theta_simplified.subs(values).evalf()

# 結果表示
sp.pprint(jacobian_ddot_theta_numeric_simplified)


⎡-221.718850794638  -115.515512609623  0  0⎤
⎢                                          ⎥
⎣704.217510196917   379.246622182072   0  0⎦


In [33]:
jacobian_ddot_theta_simplified

Matrix([
[                                           alpha10*(-alpha2 - alpha3*cos(theta2))*cos(theta1 + theta2)/(alpha2*(alpha1 + alpha4) - alpha3**2*cos(theta2)**2) + alpha2*(alpha7*cos(theta1 + theta2) - alpha8*cos(theta1))/(alpha2*(alpha1 + alpha4) - alpha3**2*cos(theta2)**2),                                                                                                                                                                                                                                                                                                          -2*alpha2*alpha3**2*(alpha5*theta2_dot**2*sin(theta2) - alpha6*theta1_dot*theta2_dot*sin(theta2) + alpha7*sin(theta1 + theta2) - alpha8*sin(theta1) + u1)*sin(theta2)*cos(theta2)/(alpha2*(alpha1 + alpha4) - alpha3**2*cos(theta2)**2)**2 + alpha2*(alpha5*theta2_dot**2*cos(theta2) - alpha6*theta1_dot*theta2_dot*cos(theta2) + alpha7*cos(theta1 + theta2))/(alpha2*(alpha1 + alpha4) - alpha3**2*cos(theta2)**2) - 2*alpha3**2*(

In [31]:
jacobian_ddot_theta

Matrix([
[                                                           0.5*G*L2*M2*(-I2 - 0.5*L1*L2*M2*cos(theta2) - 0.25*L2**2*M2)*cos(theta1 + theta2)/(-0.25*L1**2*L2**2*M2**2*cos(theta2)**2 + (I2 + 0.25*L2**2*M2)*(I1 + 0.25*L1**2*M1 + L1**2*M2)) + (I2 + 0.25*L2**2*M2)*(0.5*G*L1*M1*cos(theta1) - G*L1*M2*cos(theta1) + 0.5*G*L2*M2*cos(theta1 + theta2))/(-0.25*L1**2*L2**2*M2**2*cos(theta2)**2 + (I2 + 0.25*L2**2*M2)*(I1 + 0.25*L1**2*M1 + L1**2*M2)),                                                                                                                                                                                                                                                                                                                                                                                                                                -0.5*L1**2*L2**2*M2**2*(I2 + 0.25*L2**2*M2)*(0.5*G*L1*M1*sin(theta1) - G*L1*M2*sin(theta1) + 0.5*G*L2*M2*sin(theta1 + theta2) - L1*L2*M

In [4]:
jacobian_ddot_theta

Matrix([
[                                                           0.5*G*L2*M2*(-I2 - 0.5*L1*L2*M2*cos(theta2) - 0.25*L2**2*M2)*cos(theta1 + theta2)/(-0.25*L1**2*L2**2*M2**2*cos(theta2)**2 + (I2 + 0.25*L2**2*M2)*(I1 + 0.25*L1**2*M1 + L1**2*M2)) + (I2 + 0.25*L2**2*M2)*(0.5*G*L1*M1*cos(theta1) - G*L1*M2*cos(theta1) + 0.5*G*L2*M2*cos(theta1 + theta2))/(-0.25*L1**2*L2**2*M2**2*cos(theta2)**2 + (I2 + 0.25*L2**2*M2)*(I1 + 0.25*L1**2*M1 + L1**2*M2)),                                                                                                                                                                                                                                                                                                                                                                                                                                -0.5*L1**2*L2**2*M2**2*(I2 + 0.25*L2**2*M2)*(0.5*G*L1*M1*sin(theta1) - G*L1*M2*sin(theta1) + 0.5*G*L2*M2*sin(theta1 + theta2) - L1*L2*M